In [ ]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
import matplotlib.pyplot as plt
import numpy as np



num_qubits = 4
target_qubit = 1

qpe_circuit = QuantumCircuit(num_qubits + target_qubit, num_qubits)
qpe_circuit.h(range(num_qubits))
qpe_circuit.x(num_qubits)

for qubit in range(num_qubits):
    exponent = 2 ** qubit
    qpe_circuit.cp(2 * np.pi / exponent, qubit, num_qubits)

def qft_dagger(circuit, n):
    for qubit in range(n // 2):
        circuit.swap(qubit, n - qubit - 1)
    for j in range(n):
        for m in range(j):
            circuit.cp(-3.14159 / (2 ** (j - m)), m, j)
        circuit.h(j)

qft_dagger(qpe_circuit, num_qubits)
qpe_circuit.measure(range(num_qubits), range(num_qubits))

simulator = AerSimulator(method='statevector')
transpiled_circuit = transpile(qpe_circuit, simulator)
job = simulator.run(transpiled_circuit, shots=1024)
result = job.result()
counts = result.get_counts()

print(max(counts))

plt.bar(counts.keys(), counts.values(), color='blue')
plt.title("Measurement Histogram")
plt.show()

qpe_circuit.draw('mpl')
